CONTENT BASED RECOMMENDER SYSTEM IMPLEMENTATION 

In [1]:
#here we already have the features x, which is comprised of user features and movie features, so its diff form collaborative
#filtering in the sense that the features are already present

In [111]:
import numpy as np
import tensorflow as tf
import keras
from keras import layers
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

In [112]:
#has 397 users and 847 of movies here, here since both the numbers are not same, some of the rows from both
#user and movie dataset has been repeated so as to make them both have the same value, in this case its 50884 of them

In [113]:
#the ratings for all users for all movies are given in a singular col itself, and some of them are repeated
#as some of the movies and/or users are also repeated/boosted

In [132]:
#loading the feature data for items and users

#movie features
item = np.loadtxt('C:\\Users\\Lenovo\\OneDrive\\Desktop\\coursera_ml\\content_item_train.csv', delimiter=',')

#headers of movie features
with open('C:\\Users\\Lenovo\\OneDrive\\Desktop\\coursera_ml\\content_item_train_header.txt', 'r') as f:
    item_header = f.read().split(',')

#user features
user = np.loadtxt('C:\\Users\\Lenovo\\OneDrive\\Desktop\\coursera_ml\\content_user_train.csv', delimiter=',')

#headers of user features
with open('C:\\Users\\Lenovo\\OneDrive\\Desktop\\coursera_ml\\content_user_train_header.txt', 'r') as f:
    user_header = f.read().split(',')

#targets
y = np.loadtxt('C:\\Users\\Lenovo\\OneDrive\\Desktop\\coursera_ml\\content_y_train.csv', delimiter=',')

#unique movies
item_vecs = np.loadtxt('C:\\Users\\Lenovo\\OneDrive\\Desktop\\coursera_ml\\content_item_vecs.csv', delimiter=',')


print(user_header)
print(item_header)
print(len(item))   #50884 movies present 
print(y)
print(item.shape)
print(user.shape)

['user id', 'rating count', 'rating ave', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Horror', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller\n']
['movie id', 'year', 'ave rating', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Horror', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller\n']
50884
[4.  3.5 4.  ... 5.  5.  5. ]
(50884, 17)
(50884, 17)


In [133]:
num_user_features = user.shape[1] - 3 #removing userid, rate count, ave rating as they are not that important 
#in contributing useful data 
num_item_features = item.shape[1] - 1 #removing movieid

In [134]:
u_start = 3  #index of col from which to consider the features for users 
m_start = 1

In [135]:
#now since the data in item_train and user_train have very different scales, like some of them are small number some
#are large like years and so on hence we need to scale these features using normal scaling

#using StandardScaler for the features 
scalar_item = StandardScaler()
scalar_item.fit(item)
item_train_scaled = scalar_item.transform(item)

scalar_user = StandardScaler()
scalar_user.fit(user)
user_train_scaled = scalar_user.transform(user)

#using MinMaxScalar for the ratings or the targets, MinMaxScalar scales the numbers in the range -1 to 1, here this is
#used as it achieves the same effect. The intent of scalaing the targets was so that predicted ratings for users who have
#not rated a movie not be 0, but some other number, in this case the prediction would be w.x + b + mu where mu is the 
#avg rating of that movie for all the users, here its the similar concept where something will be added but its just
#not the mean of the movies

scaler_target = MinMaxScaler((-1,1))
scaler_target.fit(y.reshape(-1,1))
y_train_scaled = scaler_target.transform(y.reshape(-1,1))

print(y_train_scaled.shape)
print(y_train_scaled)

(50884, 1)
[[0.55555556]
 [0.33333333]
 [0.55555556]
 ...
 [1.        ]
 [1.        ]
 [1.        ]]


In [136]:
#splitting data to training and testing 
item_train, item_test = train_test_split(item_train_scaled, test_size=0.2, random_state=1, shuffle=True)
user_train, user_test = train_test_split(user_train_scaled, test_size=0.2, random_state=1, shuffle=True)
y_train, y_test = train_test_split(y, test_size=0.2, random_state=1, shuffle=True)

In [137]:
print(item_train.shape)
print(item_test.shape)
print(user_train.shape)

(40707, 17)
(10177, 17)
(40707, 17)


In [152]:
#building the NN to get vu(user vector) and vm(item vector)

tf.random.set_seed(1)

user_NN = keras.models.Sequential([
    layers.Dense(256, 'relu'),   #this is not the input layer 
    layers.Dense(128, 'relu'),
    layers.Dense(32)    #the vector vu for a single user will be outputted from this 
 ])

item_NN = keras.models.Sequential([
    layers.Dense(256, 'relu'),
    layers.Dense(128, 'relu'),
    layers.Dense(32)    #the vector vm for a single movie will be outputted from this 
 ])

input_user = layers.Input(shape=(num_user_features, ))  #input layer for user_NN, symbolic keras tensor 
vu = user_NN(input_user)   #keras tensor 
vu = tf.linalg.l2_normalize(vu, axis=1)  #making sure that mag of vu is always 1


input_item = layers.Input(shape=(num_item_features, ))  #input layer for item_NN
vm = item_NN(input_item)
# vm = tf.linalg.l2_normalize(vm, axis=1)  #making sure that mag of vu is always 1
#vm = keras.layers.UnitNormalization(axis=1)(vm)

output = layers.Dot(axes=1)([vu, vm])   #dot product of vu and vm taken in another tf layer

model = keras.Model([input_user, input_item], output)   #bunching up all the individial sequences and layers to build 
#a whole model
# A KerasTensor cannot be used as input to a TensorFlow function. "
#      93         "A KerasTensor is a symbolic placeholder for a shape and dtype, "
#      94         "used when constructing Keras Functional models "
#      95         "or Keras Functions. You can only use it as input to a Keras layer "
#      96         "or a Keras operation (from the namespaces `keras.layers` "
#      97         "and `keras.operations`). 
model.summary()


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [148]:
tf.random.set_seed(1)
cost = keras.losses.MeanSquaredError()  #instance of this class
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss=cost)


In [149]:
print(user_train[:, u_start:].shape)

(40707, 14)


In [150]:
tf.random.set_seed(1)
model.fit([user_train[:, u_start:], item_train[:, m_start:]], y_train, epochs=30, batch_size=32)

Epoch 1/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - loss: 2.2654
Epoch 2/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.6069
Epoch 3/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.6157
Epoch 4/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.6286
Epoch 5/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.6204
Epoch 6/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.6241
Epoch 7/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.6452
Epoch 8/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.6124
Epoch 9/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.6085
Epoch 10/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.5996
Epoch 11/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.5996
Epoch 12/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.5961
Epoch 13/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.5989
Epoch 14/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.5943
Epoch 15/30
1273/1273 ━━━━━━

In [142]:
#predictions

In [143]:
#creating a new user for this purpose making it the 398th user
new_user_id = 5000
new_rating_ave = 0.0
new_action = 0.0
new_adventure = 5.0
new_animation = 0.0
new_childrens = 0.0
new_comedy = 0.0
new_crime = 0.0
new_documentary = 0.0
new_drama = 0.0
new_fantasy = 5.0
new_horror = 0.0
new_mystery = 0.0
new_romance = 0.0
new_scifi = 0.0
new_thriller = 0.0
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

In [131]:
user_vecs = np.tile(user_vec, (len(item_vecs), 1))   #creating multiple rows of the same user vector for 
#847 unique movies, this is done as we will pass one row of both the user_vecs and item_vecs to the model one 
#by one for inference, and this will give us the predictes ratings for all the movies for this new user


#scaling the user data using StandardScaler
user_vecs = scalar_user.transform(user_vecs)

#scaling the movie data using StandardScaler
item_vecs = scalar_item.transform(item_vecs)

y_pred = model.predict([user_vecs[:, u_start:], item_vecs[:, m_start:]])

y_pred = scaler_target.inverse_transform(y_pred)

print(y_pred)


27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
[[-232.43585 ]
 [-231.43546 ]
 [-238.01591 ]
 [-243.55716 ]
 [-264.1941  ]
 [-209.7782  ]
 [-240.11905 ]
 [-188.16556 ]
 [-289.98874 ]
 [-257.41232 ]
 [-187.82138 ]
 [-132.50919 ]
 [-164.46262 ]
 [-272.08688 ]
 [-308.7956  ]
 [-239.12181 ]
 [-196.12042 ]
 [-138.63539 ]
 [-163.43707 ]
 [-224.16173 ]
 [-323.55392 ]
 [-232.88733 ]
 [-197.39752 ]
 [-215.01807 ]
 [-247.73422 ]
 [-301.71506 ]
 [-216.2642  ]
 [-200.32558 ]
 [-268.12064 ]
 [-274.5268  ]
 [-190.41666 ]
 [-210.89566 ]
 [-177.08493 ]
 [-247.4243  ]
 [-260.37427 ]
 [-194.86159 ]
 [-250.93607 ]
 [-233.6265  ]
 [-207.68845 ]
 [-241.91292 ]
 [ -66.6774  ]
 [-242.8829  ]
 [-240.77191 ]
 [-131.40297 ]
 [-202.47171 ]
 [-212.16086 ]
 [ -88.35124 ]
 [ -48.464222]
 [-150.05463 ]
 [-135.38661 ]
 [-297.4781  ]
 [ -79.91008 ]
 [-139.9889  ]
 [-152.4882  ]
 [-254.93518 ]
 [-211.17027 ]
 [-170.73819 ]
 [-159.40707 ]
 [-166.5516  ]
 [-232.31339 ]
 [-203.18181 ]
 [-137.28638 ]
 [-112.60663 ]
 [ -95.950676]
 